#### Задания по расчетам продуктовых метрик

[1. Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей ](#1)

[2. Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?](#2)

[3. Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)](#3)

[4. Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?](#4)

[5. ](#5)

[6. ](#6)

[7. ](#7)

[8. ](#8)

[9. ](#9)


<a id="100"></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=2, style="whitegrid", rc={'figure.figsize' : (15,7)})
import matplotlib.pyplot as plt
from scipy import special
plt.style.use('dark_background')

In [3]:
df = pd.read_csv('https://drive.usercontent.google.com/download?id=1WIR4frI8ueCN9BymYA_FAyCieCeUTddI&export=download&authuser=0&confirm=t&uuid=3a92490a-253f-48f6-b417-0431a2d24b16&at=AENtkXZolHTnvG9jdUgrCjO_tcS6%3A1733058856787')

<a id="1"></a>Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [72]:
df.date = pd.to_datetime(df.date)

In [62]:
installs = df[df.event == 'app_install']
installs

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
2515061,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads
2515062,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-
2515063,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-
2515064,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-
2515065,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal
...,...,...,...,...,...,...,...,...
2669653,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-
2669654,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-
2669655,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal
2669656,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads


In [63]:
purchases = df[df.event == 'purchase'].groupby('device_id', as_index = False).agg({'date':'min'})
purchases

,device_id,date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [67]:
df_mrg = pd.merge(installs, purchases, on = 'device_id', how = 'left', suffixes = ('_inst', '_perc'))
df_mrg

,date_inst,event,purchase_sum,os_name,device_id,gender,city,utm_source,date_perc
0,2020-01-01,app_install,NaN,ios,4921563,male,Moscow,vk_ads,2020-01-10
1,2020-01-01,app_install,NaN,ios,1311583,male,Moscow,-,NaT
2,2020-01-01,app_install,NaN,android,23006325,male,Saint-Petersburg,-,2020-01-01
3,2020-01-01,app_install,NaN,android,5645652,male,Moscow,-,NaT
4,2020-01-01,app_install,NaN,ios,8798567,male,Saint-Petersburg,referal,NaT
...,...,...,...,...,...,...,...,...,...
154592,2020-03-31,app_install,NaN,ios,19717968,female,Moscow,-,NaT
154593,2020-03-31,app_install,NaN,android,32966665,female,Saint-Petersburg,-,NaT
154594,2020-03-31,app_install,NaN,ios,6335964,female,Moscow,referal,NaT
154595,2020-03-31,app_install,NaN,android,29155826,male,Saint-Petersburg,vk_ads,NaT


In [68]:
df_mrg['int'] =df_mrg.date_perc - df_mrg.date_inst
df_mrg['int'] = df_mrg['int'].dt.days
df_mrg['active'] = (df_mrg.int < 8) & (df_mrg.int.isna() == False)
df_mrg[['date_inst', 'event','device_id', 'date_perc','int', 'active']]

,date_inst,event,device_id,date_perc,int,active
0,2020-01-01,app_install,4921563,2020-01-10,9.0,False
1,2020-01-01,app_install,1311583,NaT,NaN,False
2,2020-01-01,app_install,23006325,2020-01-01,0.0,True
3,2020-01-01,app_install,5645652,NaT,NaN,False
4,2020-01-01,app_install,8798567,NaT,NaN,False
...,...,...,...,...,...,...
154592,2020-03-31,app_install,19717968,NaT,NaN,False
154593,2020-03-31,app_install,32966665,NaT,NaN,False
154594,2020-03-31,app_install,6335964,NaT,NaN,False
154595,2020-03-31,app_install,29155826,NaT,NaN,False


In [69]:
df_mrg = df_mrg.groupby('date_inst',as_index = False).agg(inst_cnt=('event', 'count'), act_cnt=('active','sum'))
df_mrg

,date_inst,inst_cnt,act_cnt
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
...,...,...,...
86,2020-03-27,1199,277
87,2020-03-28,1091,209
88,2020-03-29,1117,223
89,2020-03-30,994,204


In [71]:
df_mrg['cr'] = df_mrg.act_cnt / df_mrg.inst_cnt
df_mrg.sort_values('cr',ascending = False)

,date_inst,inst_cnt,act_cnt,cr
0,2020-01-01,3579,1408,0.393406
8,2020-01-09,1424,558,0.391854
14,2020-01-15,4310,1650,0.382831
13,2020-01-14,5173,1973,0.381403
1,2020-01-02,3144,1186,0.377226
...,...,...,...,...
77,2020-03-18,1171,235,0.200683
88,2020-03-29,1117,223,0.199642
81,2020-03-22,1261,251,0.199048
87,2020-03-28,1091,209,0.191567


In [74]:
df.event.value_counts(dropna = False)

event
app_start      748705
search         708639
choose_item    538669
tap_basket     377665
app_install    154597
purchase       141383
register        78310
Name: count, dtype: int64

<a id="2"></a>Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?
В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».
Подсказка: CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение  

- app_install – установка приложения
- app_start – открыть приложения
- register – зарегистрироваться 
- search – перейти на страницу поиска товаров (каталог)
- open_item – открыть товар
- choose_item – отправить товар в корзину
- tap_basket – перейти в корзину
- purchase – подтверждение покупки

In [45]:
df_app_first_start = df[df.event == 'app_start'].groupby('device_id',as_index = False).agg({'date':'min'})
df_app_first_start.columns = ['dev_id','dt']
df_app_first_start.head(2)

,dev_id,dt
0,4013,2020-01-15
1,4014,2020-01-02


In [46]:
df_app_first_start = pd.merge(left = df_app_first_start, right = df[df.event == 'app_start'], how = 'left', left_on=['dev_id', 'dt'], right_on = ['device_id', 'date'])
df_app_first_start.head(2)

,dev_id,dt,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,4013,2020-01-15,2020-01-15,app_start,NaN,android,4013,female,Saint-Petersburg,vk_ads
1,4014,2020-01-02,2020-01-02,app_start,NaN,android,4014,male,Saint-Petersburg,referal


In [48]:
df_app_first_start = df_app_first_start[['date', 'event', 'device_id','utm_source']]
df_app_first_start.head(2)

,date,event,device_id,utm_source
0,2020-01-15,app_start,4013,vk_ads
1,2020-01-02,app_start,4014,referal


In [49]:
#первые установки по источнику 
df_tmp_1=df_app_first_start.groupby('utm_source',as_index=False).agg({'event':'count'})
df_tmp_1.columns=['utm_source','1st_app_start']
df_tmp_1

,utm_source,1st_app_start
0,-,41456
1,facebook_ads,18844
2,google_ads,31437
3,instagram_ads,24818
4,referal,11983
5,vk_ads,27905
6,yandex-direct,34441


In [50]:
df_first_perch = df[df.event == 'purchase'].groupby('device_id',as_index = False).agg({'date':'min'})
df_first_perch.columns = ['dev_id','dt']
df_first_perch.head(2)

,dev_id,dt
0,4014,2020-02-01
1,4046,2020-01-04


In [51]:
df_first_perch = pd.merge(left = df_first_perch, right = df[df.event == 'purchase'], how = 'left', left_on=['dev_id', 'dt'], right_on = ['device_id', 'date'])
df_first_perch = df_first_perch[['date', 'event', 'device_id','utm_source']]
df_first_perch.head(2)

,date,event,device_id,utm_source
0,2020-02-01,purchase,4014,referal
1,2020-01-04,purchase,4046,-


In [52]:
df_tmp_2=df_first_perch.groupby('utm_source',as_index=False).agg({'event':'count'})
df_tmp_2.columns=['utm_source','1st_purchase']
df_tmp_2

,utm_source,1st_purchase
0,-,14786
1,facebook_ads,7903
2,google_ads,10167
3,instagram_ads,9820
4,referal,5803
5,vk_ads,11460
6,yandex-direct,10936


In [60]:
df_tmp_3 = pd.merge(df_tmp_1, df_tmp_2, how = 'left', on = 'utm_source')
df_tmp_3['cr'] = df_tmp_3['1st_purchase'] / df_tmp_3['1st_app_start']
df_tmp_3.sort_values('cr').head(3)

,utm_source,1st_app_start,1st_purchase,cr
6,yandex-direct,34441,10936,0.317529
2,google_ads,31437,10167,0.323409
0,-,41456,14786,0.356667


<a id="3"></a>Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [64]:
df_fst_chk_date = df[df.event == 'purchase'].groupby('device_id',as_index = False).agg({'date':'min'})
df_fst_chk_date.columns = ['dev_id', 'dt']
df_fst_chk_date

,dev_id,dt
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [68]:
pd.merge(left = df_fst_chk_date, right = df[df.event == 'purchase'], how='left', left_on=['dev_id','dt'], right_on=['device_id', 'date'])\
                .groupby('utm_source', as_index = False)\
                .agg({'purchase_sum':'median'})\
                .sort_values('purchase_sum')\
                .tail(3)

,utm_source,purchase_sum
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


<a id="4"></a>Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 
ROMI = (Валовая прибыль − Маркетинговые расходы) / Маркетинговые расходы

In [78]:
#Считаем маркетинговые расходу реферальной программы - число первых покупок по рефералке * 200

ref_cost = df[(df.utm_source == '-')&(df.event == 'purchase')].groupby('device_id', as_index = False).agg({'date':'min'}).device_id.count()*200
ref_cost

np.int64(3319600)

In [81]:
#Считаем валовую прибыль по источникам
df_romi = df[df.event == 'purchase'].groupby('utm_source',as_index = False).agg({'purchase_sum':'sum'})

In [82]:
df_romi['mrkt']=[1, 8590498, 10534878, 8564626, 3319600, 9553531, 10491707]
df_romi

,utm_source,purchase_sum,mrkt
0,-,21449749.5,1
1,facebook_ads,12249901.0,8590498
2,google_ads,12868276.0,10534878
3,instagram_ads,14546969.0,8564626
4,referal,8837044.5,3319600
5,vk_ads,16389652.5,9553531
6,yandex-direct,13915368.0,10491707


In [87]:
df_romi['romi'] = round((df_romi.purchase_sum - df_romi.mrkt)/df_romi.mrkt,2)
df_romi.sort_values('romi').tail(3)

,utm_source,purchase_sum,mrkt,romi
5,vk_ads,16389652.5,9553531,0.72
4,referal,8837044.5,3319600,1.66
0,-,21449749.5,1,21449748.50
